# LiDAR Segmentation
---
This will be a notebook focusing on the LiDAR specific portions of the overall project.

First, let's start with a spherical projection to convert the 3D point clouds into a 2D projection. We will write this in Python/NumPy and then compare to a GPU accelerated version.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/")

In [4]:
import cnn
from cnn import MultiClassCrossEntropyLoss, Adam
import numpy as np
import time
import os
import glob
import zipfile

In [5]:
LOWER_FOV = np.deg2rad(-25)
UPPER_FOV = np.deg2rad(25)
VERTICAL_FOV = UPPER_FOV - LOWER_FOV

def spherical_projection(points, intensities, image_height, image_width):
  epsilon = 1e-6
  r = np.sqrt(points[:, 0]**2 + points[:, 1]**2 + points[:, 2]**2)
  r_safe = np.maximum(r, epsilon)

  theta = np.arctan2(points[:, 1], points[:, 0])
  phi = np.arcsin(points[:, 2] / r_safe)

  u = ((theta + np.pi) / (2 * np.pi)) * image_width
  v = 1 - (((phi - LOWER_FOV) / VERTICAL_FOV) * image_height)

  # Prevent image overflow
  u = np.clip(u, 0, image_width - 1)
  v = np.clip(v, 0, image_height - 1)

  u = u.astype(np.int32)
  v = v.astype(np.int32)

  projection_image = np.zeros((image_height, image_width, 5), dtype=np.float32)

  valid_indices_mask = (u >= 0) & (u < image_width) & (v >= 0) & (v < image_height)
  original_indices = np.where(valid_indices_mask)[0]

  u_valid = u[valid_indices_mask]
  v_valid = v[valid_indices_mask]
  r_valid = r[valid_indices_mask]
  points_valid = points[valid_indices_mask]
  intensities_valid = intensities[valid_indices_mask]

  # Since some points might map to the same pixel, we will keep the closest one
  sort_indices = np.argsort(r_valid)

  u_sorted = u_valid[sort_indices]
  v_sorted = v_valid[sort_indices]
  original_indices_sorted = original_indices[sort_indices]

  pixel_ids = v_sorted * image_width + u_sorted
  _, unique_indices = np.unique(pixel_ids, return_index=True)

  final_indices_in_sorted = unique_indices
  final_u = u_sorted[final_indices_in_sorted]
  final_v = v_sorted[final_indices_in_sorted]
  final_r = r_valid[sort_indices][final_indices_in_sorted]
  final_points = points_valid[sort_indices][final_indices_in_sorted]
  final_intensities = intensities_valid[sort_indices][final_indices_in_sorted]
  final_original_indices = original_indices_sorted[final_indices_in_sorted]

  final_data = np.column_stack((final_r, final_intensities, final_points))
  projection_image[final_v, final_u] = final_data

  return projection_image, final_v, final_u, final_original_indices

# Encoder-Decoder Model
---
Now that we have all of the necessary pieces, we can construct the actual model for segmentation.

We will be using a basic encoder-decoder model architecture.

In [6]:
# Semantic KITTI dataset uses 19 classes for evaluation and a 0 class for extraneous classes
NUM_CLASSES = 20

model = cnn.Sequential([
    # Encoder
    cnn.Conv2D(in_channels=5, out_channels=32, kernel_size=3, padding=1),
    cnn.ReLU(),
    cnn.MaxPool2D(pool_size=2, stride=2),

    cnn.Conv2D(in_channels=32, out_channels=64, kernel_size=3, padding=1),
    cnn.ReLU(),
    cnn.MaxPool2D(pool_size=2, stride=2),

    # Decoder
    cnn.Upsample(scale_factor=2),
    cnn.Conv2D(in_channels=64, out_channels=32, kernel_size=3, padding=1),
    cnn.ReLU(),

    cnn.Upsample(scale_factor=2),

    # Prediction layer
    cnn.Conv2D(in_channels=32, out_channels=NUM_CLASSES, kernel_size=1)
])

# Dataloader
---
We need a way to feed in the LiDAR data into our model, for this we will construct a dataloader class

In [7]:
class LidarDataset:
  def __init__(self, zip_file_path, sequences):
    self.zip_file = zipfile.ZipFile(zip_file_path, 'r')
    all_files = self.zip_file.namelist()

    self.point_cloud_files = []
    self.label_files = []

    for seq in sequences:
      pc_paths = sorted([f for f in all_files if f.startswith(f'dataset/sequences/{seq}/voxels/') and f.endswith('.bin')])
      label_paths = sorted([f for f in all_files if f.startswith(f'dataset/sequences/{seq}/voxels/') and f.endswith('.label')])

      self.point_cloud_files.extend(pc_paths)
      self.label_files.extend(label_paths)

    # We need to remap the point cloud labels into a standardized format
      self.label_remap = {
      0: 0,    # "unlabeled"
      1: 0,    # "outlier"
      10: 1,   # "car"
      11: 2,   # "bicycle"
      13: 5,   # "bus"
      15: 3,   # "motorcycle"
      16: 5,   # "on-rails"
      18: 4,   # "truck"
      20: 5,   # "other-vehicle"
      30: 6,   # "person"
      31: 7,   # "bicyclist"
      32: 8,   # "motorcyclist"
      40: 9,   # "road"
      44: 10,  # "parking"
      48: 11,  # "sidewalk"
      49: 12,  # "other-ground"
      50: 13,  # "building"
      51: 14,  # "fence"
      52: 0,   # "other-structure"
      60: 9,   # "lane-marking"
      70: 15,  # "vegetation"
      71: 16,  # "trunk"
      72: 17,  # "terrain"
      80: 18,  # "pole"
      81: 19,  # "traffic-sign"
      99: 0,   # "other-object"
      252: 1,  # "moving-car"
      253: 7,  # "moving-bicyclist"
      254: 6,  # "moving-person"
      255: 8,  # "moving-motorcyclist"
      256: 5,  # "moving-on-rails"
      257: 5,  # "moving-bus"
      258: 4,  # "moving-truck"
      259: 5   # "moving-other-vehicle"
  }

  def __len__(self):
    return len(self.point_cloud_files)

  def __getitem__(self, index):
    pc_path = self.point_cloud_files[index]
    label_path = self.label_files[index]

    pc_bytes = self.zip_file.read(pc_path)
    label_bytes = self.zip_file.read(label_path)

    # Use np.frombuffer to convert the bytes into a NumPy array
    point_cloud = np.frombuffer(pc_bytes, dtype=np.float32).reshape(-1, 4)
    labels = np.frombuffer(label_bytes, dtype=np.uint32).reshape(-1)

    points = point_cloud[:, :3]
    intensities = point_cloud[:, 3]

    image_height, image_width = 64, 1024
    projected_image, v_coords, u_coords, kept_indices = spherical_projection(
        points, intensities, image_height, image_width
    )

    projected_labels = np.zeros((image_height, image_width), dtype=np.int64)
    kept_labels = labels[kept_indices]

    remapped_labels = np.vectorize(lambda x: self.label_remap.get(x, 0))(kept_labels)
    projected_labels[v_coords, u_coords] = remapped_labels

    projected_image_transposed = projected_image.transpose(2, 0, 1).astype(np.float32)

    return projected_image_transposed, projected_labels

# Dataset
---
Alright, our dataloder handles the retrieval of the point cloud data and also converts it into a projection for us

Let's test out the model on some data, specifically the SemanticKITTI dataset.

I have uploaded the zipped training dataset (3gb) into my Drive folder already.

We will be streaming in the data batch by batch from the zipped folder as the entire dataset unzipped is about 100gb, which would be unmanageable.

# Training
---
Now that we have all of our pieces, we can finally start training.


In [8]:
DATASET_ZIP_PATH = '/content/drive/MyDrive/data_odometry_voxels_all.zip'
TRAIN_SEQUENCES = ['00'] #, '01', '02', '03', '04', '05', '06', '07']
VAL_SEQUENCES = ['08']

EPOCHS = 1
BATCH_SIZE = 4
LEARNING_RATE = 0.001

print("Initializing components...")

train_dataset = LidarDataset(zip_file_path=DATASET_ZIP_PATH, sequences=TRAIN_SEQUENCES)
val_dataset = LidarDataset(zip_file_path=DATASET_ZIP_PATH, sequences=VAL_SEQUENCES)
loss_fn = MultiClassCrossEntropyLoss()
optimizer = Adam(lr=LEARNING_RATE, beta=0.9, gamma=0.999)
print(f"All components initialized. Training on {len(train_dataset) - 4521} samples.")
print(f"All components initialized. Validating on {len(val_dataset) - 4051} samples.")

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    print(f"\n--- Starting Epoch {epoch+1}/{EPOCHS} ---")

    epoch_train_loss = 0.0
    train_batch_count = 0
    shuffled_indices = np.random.permutation(len(train_dataset))

    x_batch, y_batch = [], []
    for i in range(len(train_dataset) - 4521):
        image, label = train_dataset[shuffled_indices[i]]
        x_batch.append(image)
        y_batch.append(label)

        if len(x_batch) == BATCH_SIZE or i == len(train_dataset) - 1:
            x_batch_np = np.array(x_batch)
            y_batch_np = np.array(y_batch)

            preds = model.forward(x_batch_np)
            loss = loss_fn.forward(preds, y_batch_np)
            grad = loss_fn.backward(preds, y_batch_np)
            model.backward(grad, lambda_=0.0)
            optimizer.step(model)

            epoch_train_loss += loss
            train_batch_count += 1
            x_batch, y_batch = [], []

    avg_train_loss = epoch_train_loss / train_batch_count

    print("  Running validation...")
    epoch_val_loss = 0.0
    val_batch_count = 0

    x_val_batch, y_val_batch = [], []
    for i in range(len(val_dataset) - 4051):
        image, label = val_dataset[i]
        x_val_batch.append(image)
        y_val_batch.append(label)

        if len(x_val_batch) == BATCH_SIZE or i == len(val_dataset) - 1:
            x_val_batch_np = np.array(x_val_batch)
            y_val_batch_np = np.array(y_val_batch)

            # Get predictions and loss (No backward pass or optimizer step, it isn't necessary)
            val_preds = model.forward(x_val_batch_np)
            val_loss = loss_fn.forward(val_preds, y_val_batch_np)

            epoch_val_loss += val_loss
            val_batch_count += 1
            x_val_batch, y_val_batch = [], []

    avg_val_loss = epoch_val_loss / val_batch_count
    epoch_end_time = time.time()

    print(f"Epoch {epoch+1} Summary | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Time: {epoch_end_time - epoch_start_time:.2f}s")

print("\n--- Training Finished ---")

Initializing components...
All components initialized. Training on 20 samples.
All components initialized. Validating on 20 samples.

--- Starting Epoch 1/1 ---
  Running validation...
Epoch 1 Summary | Train Loss: 2.9727 | Val Loss: 2.9228 | Time: 98.96s

--- Training Finished ---


The focus isn't accuracy here, just to get a baseline idea of performance in both training/inference. We can see training time on CPU for just 20 samples is about 99 seconds, which is extremely slow.

Let's see how much faster GPU training is.

# Porting to PyTorch
---
Let's see how we can speed things up.

We will convert the model architecture into PyTorch, as it takes care of GPU utilization for us and a custom CUDA kernel isn't necessary here.

In [9]:
import torch
import torch.nn as nn

class LidarNet(nn.Module):
    def __init__(self):
        super(LidarNet, self).__init__()

        self.model = nn.Sequential(
            # Encoder
            nn.Conv2d(in_channels=5, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Decoder
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2),

            nn.Conv2d(in_channels=32, out_channels=NUM_CLASSES, kernel_size=1)
        )

    def forward(self, x):
        return self.model(x)

model = LidarNet()
print(model)

LidarNet(
  (model): Sequential(
    (0): Conv2d(5, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Upsample(scale_factor=2.0, mode='nearest')
    (7): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): Upsample(scale_factor=2.0, mode='nearest')
    (10): Conv2d(32, 20, kernel_size=(1, 1), stride=(1, 1))
  )
)


We will also need to convert the dataloader as well as PyTorch handles dataloading slightly differently. The PyTorch model expects PyTorch tensors instead of arrays, this can be fixed by converting numpy arrays into tensors at the end but I will rewrite it here in case I need to use the different dataloaders and also for compatibility between PyTorch operations.

In [10]:
from torch.utils.data import Dataset

class LidarDatasetPyTorch(Dataset): # Inherit from PyTorch's Dataset class
    def __init__(self, zip_file_path, sequences):
        self.zip_file = zipfile.ZipFile(zip_file_path, 'r')
        all_files = self.zip_file.namelist()
        self.point_cloud_files = []
        self.label_files = []
        for seq in sequences:
            pc_paths = sorted([f for f in all_files if f.startswith(f'dataset/sequences/{seq}/voxels/') and f.endswith('.bin')])
            label_paths = sorted([f for f in all_files if f.startswith(f'dataset/sequences/{seq}/voxels/') and f.endswith('.label')])
            self.point_cloud_files.extend(pc_paths)
            self.label_files.extend(label_paths)
            self.label_remap = {
                0: 0,    # "unlabeled"
                1: 0,    # "outlier"
                10: 1,   # "car"
                11: 2,   # "bicycle"
                13: 5,   # "bus"
                15: 3,   # "motorcycle"
                16: 5,   # "on-rails"
                18: 4,   # "truck"
                20: 5,   # "other-vehicle"
                30: 6,   # "person"
                31: 7,   # "bicyclist"
                32: 8,   # "motorcyclist"
                40: 9,   # "road"
                44: 10,  # "parking"
                48: 11,  # "sidewalk"
                49: 12,  # "other-ground"
                50: 13,  # "building"
                51: 14,  # "fence"
                52: 0,   # "other-structure"
                60: 9,   # "lane-marking"
                70: 15,  # "vegetation"
                71: 16,  # "trunk"
                72: 17,  # "terrain"
                80: 18,  # "pole"
                81: 19,  # "traffic-sign"
                99: 0,   # "other-object"
                252: 1,  # "moving-car"
                253: 7,  # "moving-bicyclist"
                254: 6,  # "moving-person"
                255: 8,  # "moving-motorcyclist"
                256: 5,  # "moving-on-rails"
                257: 5,  # "moving-bus"
                258: 4,  # "moving-truck"
                259: 5   # "moving-other-vehicle"
            }

    def __len__(self):
        return len(self.point_cloud_files)

    def __getitem__(self, index):
        pc_path = self.point_cloud_files[index]
        label_path = self.label_files[index]
        pc_bytes = self.zip_file.read(pc_path)
        label_bytes = self.zip_file.read(label_path)
        point_cloud = np.frombuffer(pc_bytes, dtype=np.float32).reshape(-1, 4)
        labels = np.frombuffer(label_bytes, dtype=np.uint32).reshape(-1)
        points = point_cloud[:, :3]
        intensities = point_cloud[:, 3]
        image_height, image_width = 64, 1024
        projected_image, v_coords, u_coords, kept_indices = spherical_projection(
            points, intensities, image_height, image_width
        )
        projected_labels = np.zeros((image_height, image_width), dtype=np.int64)
        kept_labels = labels[kept_indices]
        remapped_labels = np.vectorize(lambda x: self.label_remap.get(x, 0))(kept_labels)
        projected_labels[v_coords, u_coords] = remapped_labels
        projected_image_transposed = projected_image.transpose(2, 0, 1).astype(np.float32)

        image_tensor = torch.from_numpy(projected_image_transposed)

        label_tensor = torch.from_numpy(projected_labels)

        return image_tensor, label_tensor

In [11]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

DATASET_ZIP_PATH = '/content/drive/MyDrive/data_odometry_voxels_all.zip'
TRAIN_SEQUENCES = ['00']
VAL_SEQUENCES = ['08']
LEARNING_RATE = 0.001
BATCH_SIZE = 64
EPOCHS = 1
NUM_CLASSES = 20

train_dataset = LidarDatasetPyTorch(zip_file_path=DATASET_ZIP_PATH, sequences=TRAIN_SEQUENCES)
val_dataset = LidarDatasetPyTorch(zip_file_path=DATASET_ZIP_PATH, sequences=VAL_SEQUENCES)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = LidarNet().to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Training on {len(train_dataset)} samples.")
print(f"Validating on {len(val_dataset)} samples.")

for epoch in range(EPOCHS):
    epoch_start_time = time.time()
    print(f"\n--- Starting Epoch {epoch+1}/{EPOCHS} ---")

    model.train() # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        # Move data to the GPU
        images = images.to(device)
        labels = labels.to(device)

        # 1. Zero the gradients
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(images)

        # 3. Calculate loss
        loss = criterion(outputs, labels)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    print("  Running validation...")

    model.eval()
    running_val_loss = 0.0
    with torch.no_grad(): # We don't need to calculate gradients for validation
        for images, labels in val_loader:
            # Move data to the GPU
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

    avg_val_loss = running_val_loss / len(val_loader)
    epoch_end_time = time.time()

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Time: {epoch_end_time - epoch_start_time:.2f}s")

print("\n--- Training Finished ---")

Using device: cuda
Training on 4541 samples.
Validating on 4071 samples.

--- Starting Epoch 1/1 ---
  Running validation...
Epoch 1/1 | Train Loss: 2.1651 | Val Loss: 1.7500 | Time: 147.65s

--- Training Finished ---


# Profiling
---
Let's conduct a thorough profile of the system to see where it is stalling,  although it will be pretty obvious where it is (spherical projection).



In [12]:
!find / -name "nsys" 2>/dev/null

/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64/nsys


In [13]:
nsys_path = "/opt/nvidia/nsight-compute/2024.2.1/host/target-linux-x64"

os.environ['PATH'] = nsys_path + ':' + os.environ['PATH']

In [22]:
!nsys profile -t nvtx,cuda -o report_lidar --stats=true python train_pytorch.py > nsys_summary.txt 2>&1 # Save the log

(Profiling from a previous code run, train_pytorch from a previous script, doesn't exist anymore as I moved everything here, code left for clarity)

Looking at the profile, the total time for 1 epoch is about 150 seconds with 20 seconds being GPU compute time. This means that a little under 130 seconds of total time is spent idle/on CPU.

The bottleneck is almost surely the spherical projection as it is the most computationally intensive portion of the pipeline still being done on the CPU.

Let's write a custom kernel for the function to make it run on the GPU.


# CUDA Kernel
---


We will be using pybind11 to wrap the C++ function in python

In [15]:
!pip install pybind11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 7.1 MB/s eta 0:00:00


CoLab doesn't support C++ syntax so I will write it outside and paste it here.

In [1]:
%%writefile projection_kernel.cu
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <cuda_runtime.h>
#include <cmath> // For M_PI, sin, etc.
#include <cfloat> // For FLT_MAX

namespace py = pybind11;

// I'm getting an error about the atomicMin function on floats so
// this function uses a Compare-and-Swap loop to safely
// perform a minimum operation on floating-point numbers.

__device__ static float atomicMinFloat(float* address, float val) {
    int* address_as_int = (int*)address;
    int old = *address_as_int;
    int assumed;
    do {
        assumed = old;
        if (__int_as_float(assumed) < val) {
            break;
        }
        old = atomicCAS(address_as_int, assumed, __float_as_int(val));
    } while (assumed != old);
    return __int_as_float(old);
}

__global__ void projection_kernel(const float* points, int num_points,
                                float* range_buffer, int* index_buffer,
                                int image_height, int image_width,
                                float fov_up, float fov_down) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < num_points) {
        float x = points[index * 3 + 0];
        float y = points[index * 3 + 1];
        float z = points[index * 3 + 2];

        float r = sqrtf(x * x + y * y + z * z);
        if (r < 1e-6) return;

        float theta = atan2f(y, x);
        float phi = asinf(z / r);

        float vertical_fov = fov_up - fov_down;
        float u = (image_width - 1) * (1.0f - (theta + M_PI) / (2.0f * M_PI));
        float v = (image_height - 1) * (1.0f - (phi - fov_down) / vertical_fov);

        int u_int = static_cast<int>(roundf(u));
        int v_int = static_cast<int>(roundf(v));
        if (u_int < 0 || u_int >= image_width || v_int < 0 || v_int >= image_height) return;

        int pixel_index = v_int * image_width + u_int;

        float old_range = atomicMinFloat(&range_buffer[pixel_index], r);

        if (r < old_range) {
            index_buffer[pixel_index] = index;
        }
    }
}

__global__ void initialize_buffers_kernel(float* range_buffer, int* index_buffer, int num_elements) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < num_elements) {
        range_buffer[index] = FLT_MAX;
        index_buffer[index] = -1;
    }
}

py::array_t<int> spherical_projection_cuda(py::array_t<float, py::array::c_style | py::array::forcecast> points_py,
                                           int image_height, int image_width,
                                           float fov_up_deg, float fov_down_deg) {
    py::buffer_info points_buf = points_py.request();
    const float *points_ptr = (const float *)points_buf.ptr;
    int num_points = points_buf.shape[0];

    float *d_points, *d_range_buffer;
    int *d_index_buffer;
    size_t num_pixels = image_height * image_width;

    cudaMalloc(&d_points, num_points * 3 * sizeof(float));
    cudaMalloc(&d_range_buffer, num_pixels * sizeof(float));
    cudaMalloc(&d_index_buffer, num_pixels * sizeof(int));

    cudaMemcpy(d_points, points_ptr, num_points * 3 * sizeof(float), cudaMemcpyHostToDevice);

    int threads_per_block_init = 256;
    int blocks_per_grid_init = (num_pixels + threads_per_block_init - 1) / threads_per_block_init;
    initialize_buffers_kernel<<<blocks_per_grid_init, threads_per_block_init>>>(d_range_buffer, d_index_buffer, num_pixels);

    int threads_per_block_proj = 256;
    int blocks_per_grid_proj = (num_points + threads_per_block_proj - 1) / threads_per_block_proj;

    float fov_up_rad = fov_up_deg * M_PI / 180.0f;
    float fov_down_rad = fov_down_deg * M_PI / 180.0f;

    projection_kernel<<<blocks_per_grid_proj, threads_per_block_proj>>>(d_points, num_points, d_range_buffer, d_index_buffer, image_height, image_width, fov_up_rad, fov_down_rad);

    auto result_py = py::array_t<int>(num_pixels);
    py::buffer_info result_buf = result_py.request();
    int *result_ptr = (int *)result_buf.ptr;
    cudaMemcpy(result_ptr, d_index_buffer, num_pixels * sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_points);
    cudaFree(d_range_buffer);
    cudaFree(d_index_buffer);

    result_py.resize({image_height, image_width});
    return result_py;
}

PYBIND11_MODULE(projection_kernel, m) {
    m.def("spherical_projection", &spherical_projection_cuda, "CUDA-accelerated Spherical Projection");
}

Writing projection_kernel.cu


In [17]:
!nvcc -O3 -shared -std=c++20 -Xcompiler -fPIC `python3 -m pybind11 --includes` projection_kernel.cu -o projection_kernel`python3-config --extension-suffix`

In [18]:
!ls -l

total 2132
-rw-r--r-- 1 root root   17551 Sep 21 20:33 cnn.py
drwx------ 5 root root    4096 Sep 21 20:33 drive
-rw-r--r-- 1 root root    7999 Sep 21 20:38 nsys_summary.txt
-rwxr-xr-x 1 root root 1210592 Sep 21 20:38 projection_kernel.cpython-310-x86_64-linux-gnu.so
-rw-r--r-- 1 root root    4371 Sep 21 20:38 projection_kernel.cu
drwxr-xr-x 2 root root    4096 Sep 21 20:33 __pycache__
-rw-rw-r-- 1 root root  268147 Sep 21 20:38 report_lidar.nsys-rep
-rw-r--r-- 1 root root  651264 Sep 21 20:38 report_lidar.sqlite
drwxr-xr-x 1 root root    4096 Sep 16 13:40 sample_data


# Benchmarking
---

Alright, we have all of the pieces necessary for the final benchmarking.

Let's see how much we sped up the pipeline by.

In [19]:
import importlib.util, sys

spec = importlib.util.spec_from_file_location("projection_kernel", "./projection_kernel.cpython-310-x86_64-linux-gnu.so")
mod = importlib.util.module_from_spec(spec); spec.loader.exec_module(mod)

projection_kernel = mod

This import is included here as CoLab doesn't seem to see the compiled C++ function.

In [20]:
DATASET_ZIP_PATH = '/content/drive/MyDrive/data_odometry_voxels_all.zip'
SEQUENCES_TO_TEST = ['00', '01', '02', '03', '04', '05', '06', '07'] # Full training set

with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as archive:
    all_files = archive.namelist()
    test_files = sorted([f for f in all_files if len(f.split('/')) > 2 and f.split("/")[2] in SEQUENCES_TO_TEST and f.startswith(f'dataset/sequences/{f.split("/")[2]}/voxels/') and f.endswith('.bin')])
print(f"Found {len(test_files)} point cloud files to benchmark.")

print("\n--- Benchmarking Performance Over Full Dataset ---")
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 1024
FOV_UP_DEG = 25.0
FOV_DOWN_DEG = -25.0

# Time the Python/NumPy version over all files
print("Running Python (CPU) benchmark")
start_time_py = time.perf_counter()
with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as archive:
    for file_path in test_files:
        pc_bytes = archive.read(file_path)
        point_cloud = np.frombuffer(pc_bytes, dtype=np.float32).reshape(-1, 4)
        points_xyz = point_cloud[:, :3]
        spherical_projection(points_xyz, np.zeros(points_xyz.shape[0]), IMAGE_HEIGHT, IMAGE_WIDTH)
end_time_py = time.perf_counter()
total_time_py = end_time_py - start_time_py
print(f"Python (CPU) total time: {total_time_py:.2f} seconds")

# Time the C++/CUDA version over all files
print("\nRunning CUDA (GPU) benchmark")
start_time_cuda = time.perf_counter()
with zipfile.ZipFile(DATASET_ZIP_PATH, 'r') as archive:
    for file_path in test_files:
        pc_bytes = archive.read(file_path)
        point_cloud = np.frombuffer(pc_bytes, dtype=np.float32).reshape(-1, 4)
        points_xyz = point_cloud[:, :3].copy()
        projection_kernel.spherical_projection(points_xyz, IMAGE_HEIGHT, IMAGE_WIDTH, FOV_UP_DEG, FOV_DOWN_DEG)
end_time_cuda = time.perf_counter()
total_time_cuda = end_time_cuda - start_time_cuda
print(f"CUDA (GPU) total time:   {total_time_cuda:.2f} seconds")

speedup = total_time_py / total_time_cuda
avg_time_py_ms = (total_time_py / len(test_files)) * 1000
avg_time_cuda_ms = (total_time_cuda / len(test_files)) * 1000

print("\n--- Final Result ---")
print(f"Average time per file (Python): {avg_time_py_ms:.4f} ms")
print(f"Average time per file (CUDA):   {avg_time_cuda_ms:.4f} ms")
print(f"The custom CUDA kernel achieved a {speedup:.2f}x speedup over the full dataset.")

Found 16338 point cloud files to benchmark.

--- Benchmarking Performance Over Full Dataset ---
Running Python (CPU) benchmark
Python (CPU) total time: 187.42 seconds

Running CUDA (GPU) benchmark
CUDA (GPU) total time:   20.38 seconds

--- Final Result ---
Average time per file (Python): 11.4714 ms
Average time per file (CUDA):   1.2473 ms
The custom CUDA kernel achieved a 9.20x speedup over the full dataset.


This timing also includes the data loading step, which is insignificant

In terms of frames per second (FPS), we can assume each file to be a LiDAR sweep (frame) and we can calculate the equivalent FPS metric.

CPU: 11.4714 ms → FPS ≈ 1000 / 11.4714 ms ≈ 87.1 FPS

CUDA Accelerated: 1.2473 ms → FPS ≈ 1000 / 1.2473 ≈ 801.7 FPS

This far exceeds the typical throughput needed for LiDAR sensors, and leaves plenty of room to increase the data resolution and frequency, allowing for more accurate models.


Looks like our hunch about the spherical projection was correct.

We achieved a significant speedup using a custom CUDA kernel on the only CPU bound computational task left, the model was sped up using PyTorch and the dataloading computational load is negligible.